Esta es una primer prueba a entender el problema para el equipo de text mining. 

Considerar que todo lo redactado puede tener errores de redaccion, tipograficos, etc. porque le estoy prestando cero atencion a eso. Solo intento desglozar algunas partes del problema.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Una de las cosas mas dificiles de esta competencia es el propio dataset. El cual no existe y tenemos que construirlo nosotros de manera "creativa" usando otros data sets anteriores de competencias similares. 

A priori parece lo que lo mas inteligente es usar dos datasets (Del segundo no estoy tan convencido aun) Que son  [jigsaw-multilingual-toxic-comment-classification](http://../input/jigsaw-toxic-comment-classification-challenge) y [ruddit jigsaw dataset](http://../input/ruddit-jigsaw-dataset).

A priori voy a usar el primero para crear un dataset y ver si finalmente necesitamos algo del segundo. Super importante entender que como creemos este dataset va a impactar de manera directa en que tan buenas sean finalmente nuestras decisiones. 

**Creamos data set**

In [ ]:
#El data set a usar tiene filas con comentarios y columnas con categorias binarias. 
# Toxico, muy toxico, obseno, amenaza, insulto, discrimacion hacia la identidad, contenido sexual explicito

#ponderador
ponderador_de_toxicidad = {
    'toxic': 1,
    'severe_toxic': 2,
    'obscene': 1,
    'threat': 1,
    'insult': 1,
    'identity_hate': 2,
    'sexual_explicit': 1
}

toxicity_types = list(ponderador_de_toxicidad.keys())
toxicity_types

**Como paso siguiente deberiamos limpiar el data set**

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import pandas as pd
import numpy as np

pd.set_option('display.max_colwidth',300)
# Importo el dataset
df = pd.read_csv('../input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv', encoding='utf-8')
df.shape

In [ ]:
df.head(10)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

def get_top_n_words(corpus, n=10):
    cv = CountVectorizer()
    corpus_matrix = cv.fit_transform(corpus)
    corpus_matrix = pd.DataFrame.sparse.from_spmatrix(corpus_matrix, columns=cv.get_feature_names())
    aux = corpus_matrix.sum()
    aux=aux.to_frame('count')
    aux.sort_values(by='count',ascending=False,inplace=True)
    aux=aux.head(n)
    return aux

In [ ]:
# Cantidad de tokens en todo el corpus
import plotly.express as px

most_importants = get_top_n_words(df.comment_text, n=500)
print(most_importants.head(10))
px.bar(most_importants, x=most_importants.index, y='count')

In [ ]:
df_toxic = df[(df.toxic == 1) | (df.severe_toxic == 1) ]
df_toxic

In [ ]:
# Cantidad de tokens en todo el corpus
most_importants = get_top_n_words(df_toxic.comment_text, n=500)
print(most_importants.head(10))
px.bar(most_importants, x=most_importants.index, y='count')

**Proceso NLP**

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
stop_words = set( stopwords.words('english'))
stop_words.update(string.punctuation)
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('english')

# Pre procesamiento basico, agregar las etapas que considere necesario
def pre_procesamiento_texto(text):
  # Armo los tokens para procesar los datos
  tokens = word_tokenize(text)

  # Elimino las stopwords
  tokens = [t.lower() for t in tokens if t.lower() not in stop_words]

  return tokens

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

Tengo que crear el dataset limpio. Es decir solo con una columna de "resultado" de si es toxico o no para poder seguir probando

In [ ]:
# Combine all toxicity levels into one with the same weights set
df['toxicity'] = sum([df[type] * coef for type, coef in ponderador_de_toxicidad.items() if type in df])

# Standardize toxicity (converts to continues values)
df['toxicity'] = df['toxicity'] / df['toxicity'].max()

In [ ]:
# Downsample
df = pd.concat([
    df[df['toxicity'] <= 0].sample(n = int((df['toxicity'] > 0).sum() * 1.5),random_state = 201),
    df[df['toxicity'] > 0]
])
print(f"- New shape: {df.shape}")
df

In [ ]:
# Vectorizacion de resultados BOW, tambien se puede probar TF-IDF
cv = CountVectorizer(tokenizer=pre_procesamiento_texto, max_df=0.2, min_df=0.03)
X_transform_cv = cv.fit_transform(df.comment_text) # Armo matriz para entrenar CV

# TF-IDF 
tf = TfidfVectorizer(tokenizer=pre_procesamiento_texto, max_df=0.2, min_df=0.03)
X_transform_tf = tf.fit_transform(df.comment_text) # Armo matriz para entrenar tf
print(X_transform_cv.shape)
print(X_transform_tf.shape)

In [ ]:
#Separado X,y en train y test, dos veces porque cada proceso de NLP es distinto

X_traincv, X_testcv, y_train, y_test = train_test_split(X_transform_cv, df.toxicity,test_size=.30)
X_traintf, X_testtf, y_train, y_test = train_test_split(X_transform_tf, df.toxicity,test_size=.30)
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')


In [ ]:
df_transform = pd.DataFrame.sparse.from_spmatrix(X_traincv, columns=cv.get_feature_names())


In [ ]:
aux = df_transform.sum().to_frame('count')
aux.sort_values(by='count',ascending=False,inplace=True)
aux=aux.head(100)
px.bar(aux, x=aux.index, y='count')

**Primer modelo**

In [ ]:


## CV

# Entreno CV - regresión lineal
from sklearn.linear_model import LinearRegression
learner1 = LinearRegression()
learner1.fit(X_traincv, y_train)



In [ ]:
"""""
Lightlg
import lightgbm as lgb
learner5 = lgb.LGBMRegressor(n_estimators=1000, max_depth=10)
learner5.fit(X_traintf, y_train)
"""""

In [ ]:
## CV

#Entreno CV - Random Forest
from sklearn.ensemble import RandomForestRegressor
learner2 = RandomForestRegressor(n_estimators = 100, random_state = 0)
learner2.fit(X_traincv, y_train)

In [ ]:
## TF

# Entreno TF - Regresion
from sklearn.linear_model import LinearRegression
learner3 = LinearRegression()
learner3.fit(X_traintf, y_train)

In [ ]:
## TF

# Entreno TF - Random Forest
from sklearn.linear_model import LinearRegression
learner4 = RandomForestRegressor(n_estimators = 100, random_state = 0)
learner4.fit(X_traintf, y_train)

In [ ]:
#Hago predicciones sobre Test

## Para CV Regresion
y_pred1 = learner1.predict(X_testcv)

## Para CV RF
y_pred2 = learner2.predict(X_testcv)

## Para CV Regresion
y_pred3 = learner3.predict(X_testcv)

## Para CV RF
y_pred4 = learner4.predict(X_testcv)



In [ ]:
# RMSE (Root Mean Square Error)

from sklearn.metrics import mean_squared_error 

rmse1 = float(format(np.sqrt(mean_squared_error(y_test, y_pred1)),'.3f'))
print("\nRMSE - CV Reg Mult:\n",rmse1)
rmse2 = float(format(np.sqrt(mean_squared_error(y_test, y_pred2)),'.3f'))
print("\nRMSE - CV Random Forest:\n",rmse2)
rmse3 = float(format(np.sqrt(mean_squared_error(y_test, y_pred3)),'.3f'))
print("\nRMSE - TF Reg Mult:\n",rmse3)
rmse4 = float(format(np.sqrt(mean_squared_error(y_test, y_pred4)),'.3f'))
print("\nRMSE - TF Random Forest:\n",rmse4)

In [ ]:
## Bert

import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
df2 = df
df2.head(2)

In [ ]:
df2.rename(columns = {'toxicity':'Category', 'comment_text':'Message'}, inplace = True)
df2.head(3)

In [ ]:
df2.drop(['id','toxic', 'severe_toxic', 'obscene','threat','insult','identity_hate'], axis = 'columns', inplace=True)
df2.head(5)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df2['Message'],df2['Category'], stratify=df['Category'])

In [ ]:
!pip install tensorflow-text

In [ ]:
import tensorflow_text as text

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

get_sentence_embeding([
    "500$ discount. hurry up", 
    "Bhavin, are you up for a volleybal game tomorrow?"]
)

In [ ]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=1, batch_size = 32)

In [ ]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()